In [1]:
import os
assert os.environ['CONDA_DEFAULT_ENV'] == 'webscrape', "THE CONDA ENVIRONMENT MUST BE WEBSCARPE"

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime
import numpy as np

In [2]:
def load_login_details():
    with open('logins.json') as f:
        details = json.load(f)
    return details['username'], details['password']

def random_wait(mult=7):
    time.sleep(round(np.random.random() * mult, 5))

In [3]:
def open_timetable_page(driver):
    USERNAME, PASSWORD = load_login_details()
    # load up first page
    driver.get("https://www.rmit.edu.au/students/student-essentials/class-timetables")
    random_wait()    
    # click "Log in to myTimetable
    driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/section[2]/div/div[1]/div[2]/div/a').click()
    random_wait()   

    # log in elements
    usernameElement = driver.find_element_by_xpath('/html/body/div[10]/div[1]/div/form/div/div[1]/div/input')
    usernameElement.clear()
    usernameElement.send_keys(USERNAME)

    passwordElement = driver.find_element_by_xpath('/html/body/div[10]/div[1]/div/form/div/div[2]/div/input')
    passwordElement.clear()
    passwordElement.send_keys(PASSWORD)

    random_wait()
    
    # perform log in
    driver.find_element_by_xpath('/html/body/div[10]/div[1]/div/form/div/div[3]/span').click()
    random_wait()



In [4]:
driver = webdriver.Chrome("C:\webdrivers\chromedriver.exe")
open_timetable_page(driver)




In [5]:
def find_with_soup(page_source, classType):
    soup = BeautifulSoup(page_source, 'html')
    table = soup.find('table', attrs={'class':'aplus-table'})
    mainBody = table.find('tbody')
    usefulRows = mainBody.find_all('tr')
    # Assuming table format Activity, Day, Time, Campus, Location, Duration, Weeks, Description
    for i in range(len(usefulRows)):
        usefulRows[i] = usefulRows[i].text.split('\n')
    
    duration = float(usefulRows[0][8].split(" ")[0])
    location = usefulRows[0][6]
    classTimes = []
    day_map = {
        "Mon":1,
        "Tue":2,
        "Wed":3,
        "Thu":4,
        "Fri":5,
        "Sat":6,
        "Sun":7
    }

    for j in range(len(usefulRows)):
        classTimes.append({
            "day_no":day_map[usefulRows[j][4]],
            "start_time":usefulRows[j][5],
            "weeks":usefulRows[j][9],
            "availability":usefulRows[j][1]
            })


    return {"type":classType, "duration":duration, "location":location, "times":classTimes}
    







In [7]:
# element that holds the subject sidebar
subjectList = driver.find_element_by_xpath('/html/body/div[4]/div/div[2]/div[2]/div/div[2]/div/div[2]/ul')
elements = subjectList.find_elements_by_tag_name('li')
# go through following li elemets 
counter = 0
current_code = "None"
current_course = "None"
data = {"courses":[]}
for i in range(len(elements)):
    elem = elements[i]
    if elem.get_attribute('class') == 'subject':
        # get all the text
        counter += 1
        print('title', counter)
        elemText = elem.text.split('\n')
        current_code = elemText[0].split('_')[0] # get only the code part
        current_course = elemText[1] # get the course name
        data["courses"].append({
            "name":current_course,
            "code":current_code,
            "lessons":[]
            })
       
    elif elem.get_attribute('class') == 'action':
        # go into the course's information
        elem.click()
        counter += 1
        print('start click', counter)
        classType = elem.get_attribute("data-group")[:-2]
        print(classType)
        data["courses"][-1]["lessons"].append(find_with_soup(driver.page_source, classType))
        print('done click')
        random_wait(2)
        
    else:
        raise ValueError

# data has now been created
# time to save it
with open('lessonOptions.json', "w") as f:
    json.dump(data, f, indent=4)
print('finished downloading timetable\n')
driver.close()
        

title 1
start click 2
LTL
done click
start click 3
PRA
done click
start click 4
TUT
done click
title 5
start click 6
LTL
done click
start click 7
PRA
done click
start click 8
TUT
done click
title 9
start click 10
TUT
done click
title 11
start click 12
LTL
done click
start click 13
PRA
done click
start click 14
TUT
done click
finished downloading timetable

